In [1]:
import re
import numpy as np

with open("input/day16_input.txt", "r") as f:
    notes = [l.rstrip() for l in f.readlines()]

In [2]:
re_exp = r"^([\w\s]+):\s(\d+)-(\d+)\sor\s(\d+)-(\d+)"
my_ticket_idx = notes.index("your ticket:")
nearby_ticket_idx = notes.index("nearby tickets:")

In [3]:
rules = [re.findall(re_exp, line)[0] for line in notes[:my_ticket_idx - 1]]
rules = {rule[0]: [int(r) for r in rule[1:]] for rule in rules}
my_ticket = list(map(int, notes[my_ticket_idx + 1].split(',')))
nearby_tickets = [list(map(int, notes[i].split(','))) for i in range(nearby_ticket_idx + 1, len(notes))]

In [4]:
def check_error_rate():
    errors = []
    invalid_tickets = []

    for i, ticket in enumerate(nearby_tickets):
        for f in ticket:
            flag = 0
            for _, r in rules.items():
                if not (r[0] <= f <= r[1] or r[2] <= f <= r[3]):
                    flag += 1

            if flag == len(rules):
                errors.append(f)
                invalid_tickets.append(i)

    return sum(errors), invalid_tickets

In [5]:
error_rate, invalid_tickets = check_error_rate()
error_rate

20013

# Part 2

In [6]:
nearby_tickets = np.delete(np.array(nearby_tickets), invalid_tickets, axis=0)
nearby_tickets

array([[446, 499, 748, ..., 185, 828, 423],
       [875, 895, 652, ..., 716, 887, 674],
       [899, 387, 218, ..., 286, 628, 571],
       ...,
       [415,  54, 814, ...,  91, 879, 844],
       [119, 457, 222, ..., 406, 326, 848],
       [591, 319, 492, ..., 936, 127, 511]])

In [7]:
def check_rule(col, r):
    return np.all((col >= r[0]) & (col <= r[1]) | (col >= r[2]) & (col <= r[3]))

def check_candidates(candidates):
    column_count = {k: [] for k in range(len(my_ticket))}
    for col_idx in range(len(my_ticket)):
        col = nearby_tickets[:, col_idx]
        for rule_name, rule_values in rules.items():
            if rule_name in candidates and check_rule(col, rule_values):
                column_count[col_idx].append(rule_name)
                
    for k, v in column_count.items():
        if len(v) == 1:
            return k, v[0]

In [8]:
mappings = {}
candidates = list(rules.keys())

while len(candidates) > 0:
    col, field = check_candidates(candidates)
    mappings[field] = col
    candidates.remove(field)

print(mappings)
departure_values = [my_ticket[v] for k, v in mappings.items() if k.startswith("departure")]       
np.product(departure_values)

{'row': 12, 'arrival station': 11, 'price': 6, 'seat': 8, 'route': 2, 'type': 18, 'arrival track': 13, 'departure station': 0, 'departure date': 7, 'departure location': 4, 'departure track': 17, 'departure time': 10, 'departure platform': 16, 'class': 1, 'zone': 15, 'arrival platform': 5, 'duration': 14, 'wagon': 9, 'arrival location': 3, 'train': 19}


5977293343129